# Inicialização

In [8]:
# Dependências gerais
import numpy as np

# Dependências locais
from utils.glfw_window import GLFWWindow
from utils.opengl_session import OpenGLSession
from utils.colors import rgb_black_color, rgb_random_color
from utils.components import *
from utils.transformations import *

# Propriedades da janela
WINDOW_WIDTH = 720
WINDOW_HEIGHT = 480
WINDOW_TITLE = "Fazendinha Contente"

# Inicialização da janela e da sessão
window = GLFWWindow(WINDOW_WIDTH, WINDOW_HEIGHT, WINDOW_TITLE)
session = OpenGLSession(window, num_dims=2)

# Componentes

In [9]:
# Cenário Estático
scenario = OpenGLComponentCompound ([

    # Céu e grama
    Rectangle(width = WINDOW_WIDTH, height = WINDOW_HEIGHT, color = [0.6, 0.8, 0.9, 1.0]),
    Rectangle(x0 = -0.8, y0 = -0.8, width = WINDOW_WIDTH, height = 1.0, color = [0.0, 1.0, 0.0, 1.0]),

    # Base e topo do Moinho
    Rectangle(x0 = -0.55, y0 = -0.1, width = 0.35, height = 0.85, color = [0.5, 0.5, 0.5, 1.0]),
    Triangles(vertices = np.array([(-0.73, 0.32), (-0.37, 0.32), (-0.54, 0.56)]), color = [0.7, 0.5, 0.5, 1.0]),
    
    # Casa, telhado, porta, maçaneta, janela
    Square(x0 = -0.7, y0 = -0.7, side = 0.5, color = [0.58, 0.29, 0.1, 1.0]),
    Triangles(vertices = np.array([(-0.95, -0.45), (-0.70, -0.1), (-0.45, -0.45)]), color = [0.78, 0.38, 0.35, 1.0]),
    Rectangle(x0 = -0.55, y0 = -0.75, width = 0.1, height = 0.4, color = [0.72, 0.75, 0.9, 1.0]),
    Circle(x0 = -0.52, y0 = -0.75, radius = 0.01, color = rgb_black_color()),
    Square(x0 = -0.8, y0 = -0.65, side = 0.2, color = [0.68, 0.93, 0.93, 1.0]),
    Lines(vertices = np.array([(-0.9, -0.65), (-0.7, -0.65)]), color = rgb_black_color()),
    Lines(vertices = np.array([(-0.8, -0.55), (-0.8, -0.75)]), color = rgb_black_color()),

    # Locais dos animais
    Rectangle(x0 = -0.3, y0 = -0.65, width = 0.01, height = 0.4, color = [0.58, 0.29, 0.1, 1.0]),
    Rectangle(x0 = -0.1, y0 = -0.65, width = 0.01, height = 0.4, color = [0.58, 0.29, 0.1, 1.0]),
    Rectangle(x0 = -0.2, y0 = -0.45, width = 0.2, height = 0.01, color = [0.58, 0.29, 0.1, 1.0]),

    Rectangle(x0 = 0.1, y0 = -0.65, width = 0.01, height = 0.4, color = [0.58, 0.29, 0.1, 1.0]),
    Rectangle(x0 = 0.3, y0 = -0.65, width = 0.01, height = 0.4, color = [0.58, 0.29, 0.1, 1.0]),
    Rectangle(x0 = 0.2, y0 = -0.45, width = 0.2, height = 0.01, color = [0.58, 0.29, 0.1, 1.0]),
    
    Rectangle(x0 = 0.5, y0 = -0.65, width = 0.01, height = 0.4, color = [0.58, 0.29, 0.1, 1.0]),
    Rectangle(x0 = 0.7, y0 = -0.65, width = 0.01, height = 0.4, color = [0.58, 0.29, 0.1, 1.0]),
    Rectangle(x0 = 0.6, y0 = -0.45, width = 0.2, height = 0.01, color = [0.58, 0.29, 0.1, 1.0]),

    # Sol
    Circle(x0 = 0.75, y0 = 0.75, radius = 0.2, color = [1.0, 1.0, 0.0, 1.0]),
    
    # Raios
    Lines(vertices = np.array([(0.75, 0.75), (1.0, 0.9)]), color = [1.0, 1.0, 0.0, 1.0]),
    Lines(vertices = np.array([(0.75, 0.75), (1.0, 0.5)]), color = [1.0, 1.0, 0.0, 1.0]),
    Lines(vertices = np.array([(0.75, 0.75), (0.75, 0.4)]), color = [1.0, 1.0, 0.0, 1.0]),
    Lines(vertices = np.array([(0.75, 0.75), (0.5, 0.5)]), color = [1.0, 1.0, 0.0, 1.0]),
    Lines(vertices = np.array([(0.75, 0.75), (0.45, 0.9)]), color = [1.0, 1.0, 0.0, 1.0]),
    Lines(vertices = np.array([(0.75, 0.75), (0.75, 1.0)]), color = [1.0, 1.0, 0.0, 1.0]),

    # Tronco árvore
    Rectangle(x0 = 0.87, y0 = -0.45, width = 0.13, height = 0.5, color = [0.58, 0.29, 0.1, 1.0]),
    
    # Folhas
    Circle(x0 = 0.9, y0 = -0.2, radius = 0.1, color = [0.0, 0.9, 0.1, 1.0]),
    Circle(x0 = 0.82, y0 = -0.2, radius = 0.1, color = [0.0, 0.9, 0.1, 1.0]),
    Circle(x0 = 0.92, y0 = -0.1, radius = 0.1, color = [0.0, 0.9, 0.1, 1.0]),
    Circle(x0 = 0.8, y0 = -0.07, radius = 0.1, color = [0.0, 0.9, 0.1, 1.0]),
    
    # Maças
    Circle(x0 = 0.86, y0 = -0.15, radius = 0.02, color = [1.0, 0.0, 0.1, 1.0]),
    Circle(x0 = 0.79, y0 = -0.1, radius = 0.02, color = [1.0, 0.0, 0.1, 1.0]),
    Circle(x0 = 0.9, y0 = -0.05, radius = 0.02, color = [1.0, 0.0, 0.1, 1.0]),
    Circle(x0 = 0.93, y0 = -0.2, radius = 0.02, color = [1.0, 0.0, 0.1, 1.0]),
    Circle(x0 = 0.77, y0 = -0.25, radius = 0.02, color = [1.0, 0.0, 0.1, 1.0])
])

# Moinho
moinho = OpenGLComponentCompound ([
    
    # Centro
    Circle(x0 = -0.54, y0 = 0.56, radius = 0.04, color = [0.7, 0.1, 0.4, 1.0]),
    
    # Pás
    Rectangle(x0 = -0.3, y0 = 0.56, width = 0.35, height = 0.15, color = [0.6, 0.2, 0.7, 1.0]),
    Rectangle(x0 = -0.78, y0 = 0.56, width = 0.35, height = 0.15, color = [0.6, 0.2, 0.7, 1.0]),
    Rectangle(x0 = -0.54, y0 = 0.84, width = 0.12, height = 0.4, color = [0.6, 0.2, 0.7, 1.0]),
    Rectangle(x0 = -0.54, y0 = 0.28, width = 0.12, height = 0.4, color = [0.6, 0.2, 0.7, 1.0]),
    
    # Conexão centro e pás
    Lines(vertices = np.array([(-0.54, 0.56), (-0.61, 0.56)]), color = rgb_black_color()),
    Lines(vertices = np.array([(-0.54, 0.56), (-0.47, 0.56)]), color = rgb_black_color()),
    Lines(vertices = np.array([(-0.54, 0.56), (-0.54, 0.65)]), color = rgb_black_color()),
    Lines(vertices = np.array([(-0.54, 0.56), (-0.54, 0.47)]), color = rgb_black_color())
])

# Nave
nave = OpenGLComponentCompound ([

    # Luzes Inferiores
    Circle(x0 = 0.53, y0 = 0.65, radius = 0.01, color = [0.07, 0.67, 0.51, 1.0]),
    Circle(x0 = 0.6, y0 = 0.65, radius = 0.01, color = [0.07, 0.67, 0.51, 1.0]),
    Circle(x0 = 0.67, y0 = 0.65, radius = 0.01, color = [0.07, 0.67, 0.51, 1.0]),
    Circle(x0 = 0.74, y0 = 0.65, radius = 0.01, color = [0.07, 0.67, 0.51, 1.0]),
    Circle(x0 = 0.81, y0 = 0.65, radius = 0.01, color = [0.07, 0.67, 0.51, 1.0]),
    Circle(x0 = 0.88, y0 = 0.65, radius = 0.01, color = [0.07, 0.67, 0.51, 1.0]),
    Circle(x0 = 0.96, y0 = 0.65, radius = 0.01, color = [0.07, 0.67, 0.51, 1.0]),

    # Estrutura principal
    Triangles(vertices = np.array([(0.75, 0.8), (0.50, 0.65), (1.0, 0.65)]), color = rgb_black_color()),

    # Luzes Superiores
    Circle(x0 = 0.65, y0 = 0.72, radius = 0.01, color = [0.07, 0.67, 0.51, 1.0]),
    Circle(x0 = 0.74, y0 = 0.71, radius = 0.01, color = [0.07, 0.67, 0.51, 1.0]),
    Circle(x0 = 0.83, y0 = 0.72, radius = 0.01, color = [0.07, 0.67, 0.51, 1.0]),
    Circle(x0 = 0.75, y0 = 0.8, radius = 0.01, color = [0.07, 0.67, 0.51, 1.0])
])

# MIMOSA, a vaca
mimosa = OpenGLComponentCompound ([

    # Patas
    Square(x0=0.2, y0=0.1, side=0.2, color=rgb_random_color()),
    Square(x0=0.6, y0=0.1, side=0.2, color=rgb_random_color()),

    # Corpo e cauda
    Rectangle(x0=0.4, y0=0.35, width=0.6, height=0.3, color=rgb_random_color()),
    Triangles(vertices=np.array([(0.0, 0.5), (0.1, 0.5), (0.1, 0.6)]), color=rgb_random_color()), 

    # Cabeça, olhos e chifres
    Square(x0=0.6, y0=0.6, side=0.2, color=rgb_random_color()),
    Circle(x0=0.55, y0=0.65, radius=0.03, color=rgb_random_color()), 
    Circle(x0=0.65, y0=0.65, radius=0.03, color=rgb_random_color()), 
    Triangles(vertices=np.array([(0.4, 0.7), (0.5, 0.7), (0.5, 0.8)]), color=rgb_random_color()), 
    Triangles(vertices=np.array([(0.7, 0.7), (0.8, 0.7), (0.7, 0.8)]), color=rgb_random_color()), 
])

# CLÓVIS, o porco
clovis = OpenGLComponentCompound ([

    # Patas
    Square(x0=0.2, y0=0.1, side=0.2, color=rgb_random_color()),
    Square(x0=0.6, y0=0.1, side=0.2, color=rgb_random_color()),

    # Corpo e cauda
    Rectangle(x0=0.4, y0=0.35, width=0.6, height=0.3, color=rgb_random_color()),
    Triangles(vertices=np.array([(0.0, 0.5), (0.1, 0.5), (0.1, 0.6)]), color=rgb_random_color()), 

    # Cabeça, olhos e chifres
    Square(x0=0.6, y0=0.6, side=0.2, color=rgb_random_color()),
    Circle(x0=0.55, y0=0.65, radius=0.03, color=rgb_random_color()), 
    Circle(x0=0.65, y0=0.65, radius=0.03, color=rgb_random_color()),
    Circle(x0 = 0.60, y0 = 0.6, radius = 0.02, color = rgb_random_color()),
    Triangles(vertices=np.array([(0.4, 0.7), (0.5, 0.7), (0.5, 0.6)]), color=rgb_random_color()), 
    Triangles(vertices=np.array([(0.7, 0.7), (0.8, 0.7), (0.7, 0.6)]), color=rgb_random_color()), 
])

# CARAMELO, o bezerro
caramelo = OpenGLComponentCompound ([

    # Patas
    Square(x0=0.2, y0=0.1, side=0.2, color=rgb_random_color()),
    Square(x0=0.6, y0=0.1, side=0.2, color=rgb_random_color()),

    # Corpo e cauda
    Rectangle(x0=0.4, y0=0.35, width=0.6, height=0.3, color=rgb_random_color()),
    Rectangle(x0 = 0.05, y0 = 0.45, width = 0.1, height = 0.05, color = rgb_random_color()),

    # Cabeça, olhos e chifres
    Square(x0=0.6, y0=0.6, side=0.2, color=rgb_random_color()),
    Circle(x0=0.55, y0=0.65, radius=0.03, color=rgb_random_color()), 
    Circle(x0=0.65, y0=0.65, radius=0.03, color=rgb_random_color()), 
    Triangles(vertices=np.array([(0.4, 0.7), (0.5, 0.7), (0.45, 0.65)]), color=rgb_random_color()), 
    Triangles(vertices=np.array([(0.7, 0.7), (0.8, 0.7), (0.75, 0.65)]), color=rgb_random_color()), 
])

# Adição dos componentes criados anteriormente na sessão
session.add_components((scenario, moinho, nave, mimosa))

# Eventos de teclado e de mouse

In [10]:
# Transformação da Mimosa
def transform_mimosa(mimosa):

    # Parâmetros
    tx = 0.0
    ty = 0.0
    sx = 1.0
    sy = 1.0
    t = 0.0
    def closure(*args):

        # Atualização dos parâmetros
        nonlocal tx, ty, sx, sy, t, mimosa
        tx += 0.001
        ty += 0.001
        sx -= 0.01
        sy -= 0.01
        t += np.pi/8

        # Matrizes
        trans = translate3D(tx, ty)
        scale = scale3D(sx, sy)
        rotat = relative_zrotate3D(t, x0=0.35, y0=0.4)

        # Transformação
        mimosa.transformation = np.linalg.multi_dot((trans, scale, rotat))
        
    return closure

# Transformação do Clóvis
def transform_clovis(clovis):

    # Parâmetros
    tx = 0.0
    ty = 0.0
    sx = 1.0
    sy = 1.0
    t = 0.0
    def closure(*args):

        # Atualização dos parâmetros
        nonlocal tx, ty, sx, sy, t, clovis
        tx += 0.001
        ty += 0.001
        sx -= 0.01
        sy -= 0.01
        t += np.pi/8

        # Matrizes
        trans = translate3D(tx, ty)
        scale = scale3D(sx, sy)
        rotat = relative_zrotate3D(t, x0=0.35, y0=0.4)

        # Transformação
        mat = np.dot(trans, scale)
        mat = np.dot(mat, rotat)
        clovis.transformation = mat
        
    return closure

# Transformação do Caramelo
def transform_caramelo(caramelo):

    # Parâmetros
    tx = 0.0
    ty = 0.0
    sx = 1.0
    sy = 1.0
    t = 0.0
    def closure(*args):

        # Atualização dos parâmetros
        nonlocal tx, ty, sx, sy, t, caramelo
        tx += 0.001
        ty += 0.001
        sx -= 0.01
        sy -= 0.01
        t += np.pi/8

        # Matrizes
        trans = translate3D(tx, ty)
        scale = scale3D(sx, sy)
        rotat = relative_zrotate3D(t, x0=0.35, y0=0.4)

        # Transformação
        mat = np.dot(trans, scale)
        mat = np.dot(mat, rotat)
        caramelo.transformation = mat
        
    return closure

# Transformação do moinho
def transform_moinho(moinho):
    
    # Parâmetros
    t = 0.0
    def closure(*args):

        # Atualização dos parâmetros
        nonlocal t, moinho
        t += np.pi/8

        # Matriz
        rotat = relative_zrotate3D(t, x0 = -0.54, y0 = 0.56)

        # Transformação
        moinho.transformation = rotat
        
    return closure

# Transformação da nave espacial
def transform_nave(nave, op:int = 0):

    # Parâmetros
    tx = 0
    ty = 0
    def closure(*args):

        # Atualização dos parâmetros
        nonlocal tx, ty, op, nave
        ty += 0.01 if op == 1 else 0.0
        ty -= 0.01 if op == 2 else 0.0
        tx += 0.01 if op == 3 else 0.0
        tx -= 0.01 if op == 4 else 0.0

        # Matriz
        trans = translate3D(tx, ty)

        # Transformação
        nave.transformation = trans
        
    return closure

# Atribuição dos eventos de tecla
window.add_key_callback("M", transform_mimosa(mimosa))
window.add_key_callback("K", transform_caramelo(caramelo))
window.add_key_callback("P", transform_clovis(clovis))
window.add_key_callback("V", transform_moinho(moinho))
window.add_key_callback("W", transform_nave(nave, 1))
window.add_key_callback("S", transform_nave(nave, 2))
window.add_key_callback("D", transform_nave(nave, 3))
window.add_key_callback("A", transform_nave(nave, 4))

# Execução

In [11]:
# Processamento dos buffers
session.process_buffers()

# Programa principal
for _ in window.display():

    # Limpeza da janela
    session.clear()

    # Renderização
    session.render()